# 金融 证券 图谱搭建

本项目为基于 Python 中金融数据包 `TUSHARE` ，搭建一个金融证券知识图谱。

![](../pictures/structures.png)

## 数据获取

数据来源：TUSHARE

网站：https://tushare.pro/document/2

从 `TUSHARE` 的接口文档，对数据进行筛选。找出关联数据，这里我先选择以下**6**类数据。
- 股票列表
- 上市公司基本信息
- 上市公司管理层
- 公募基金列表
- 公募基金公司
- 公募基金持仓数据



In [7]:
import tushare as ts
import pandas as pd

from datetime import timedelta,datetime
import time

In [12]:
ts.set_token('8b95d452f05b5041d7b6bc0de70753d69ce9c347cc2821afb4e5b3a8')
pro = ts.pro_api()

In [18]:
data_path = '../data/'

### 获取上市公司名录

链接：https://tushare.pro/document/2?doc_id=25

|名称|类型|描述|
|--|--|--|
|ts_code|str|TS代码|
|symbol|str|<font  color=#009688  >**股票代码**</font>|
|name|str|<font  color=#009688  >**股票名称**</font>|
|area|str|所在地域|
|industry|str|<font  color=#009688  >**所属行业**</font>|
|fullname|str|股票全称|
|enname|str|英文全称|
|market|str|市场类型 （主板/中小板/创业板/科创板）|
|exchange|str|交易所代码|
|curr_type|str|交易货币|
|list_status|str|上市状态： L上市 D退市 P暂停上市|
|list_date|str|上市日期|
|delist_date|str|退市日期|
|is_hs|str|是否沪深港通标的，N否 H沪股通 S深股通|

In [19]:
columns = 'ts_code,symbol,name,area,industry,fullname,enname,market,exchange,curr_type,list_status,list_date,delist_date,is_hs'

total_stocks = pro.stock_basic(fields = columns)
# total_stocks = pro.query('stock_basic', exchange='', list_status='L', fields= columns)

total_stocks.shape

(3789, 14)

total_stocks.exchange.value_counts()

In [20]:
# 保存数据
total_stocks.to_csv(data_path + 'company_lsit_all.csv')

In [21]:
total_stocks = pd.read_csv(data_path + 'company_lsit_all.csv')

In [22]:
total_stocks.market.value_counts()

主板     1958
中小板     945
创业板     795
科创板      91
Name: market, dtype: int64

### 上市公司基本信息

这部分数据与上面的上市公司名录互补，提取所在省份和城市。

|名称|类型|默认显示|描述|
|---|---|---|---|
|ts_code|str|Y| <font  color=#009688  >**股票代码**</font>|
|exchange|str|Y| 交易所代码 ，SSE上交所 SZSE深交所|
|chairman|str|Y| 法人代表|
|manager |str|Y| 总经理|
|secretary| str|Y| 董秘|
|reg_capital |float| Y| 注册资本|
|setup_date|str|Y| 注册日期|
|province|str|Y| <font  color=#009688  >**所在省份**</font>|
|city|str|Y|  <font  color=#009688  >**所在城市**</font>|
|introduction|str|N| 公司介绍|
|website |str|Y| 公司主页|
|email| str|Y| 电子邮件|
|office|str|N| 办公室|
|employees| int|Y| 员工人数|
|main_business| str|N| 主要业务及产品|
|business_scope|str|N| 经营范围|

In [23]:
df_szse =  pro.stock_company(exchange = "SZSE")
df_sse =  pro.stock_company(exchange = "SSE")

# 合并 深交所 和 上交所 股票
df_company_detail = pd.concat([df_szse,df_sse])

In [24]:
df_szse.shape,df_sse.shape,df_company_detail.shape

((2273, 12), (1651, 12), (3924, 12))

In [26]:
# 保存数据
df_company_detail.to_csv('../data/company_detail_all.csv',index=False)

In [27]:
df_company_detail = pd.read_csv('../data/company_detail_all.csv')

In [28]:
df_company_detail.head().T

,0,1,2,3,4
ts_code,300381.SZ,300222.SZ,300416.SZ,300234.SZ,300395.SZ
exchange,SZSE,SZSE,SZSE,SZSE,SZSE
chairman,陈少美,黄明松,钟琼华,邢翰学,吴学民
manager,陈少美,黄明松,钟琼华,邢翰学,商春利
secretary,周德荣,穆峻柏,陈英,许哲远,郑巍
reg_capital,43984.2,72475.1,13557.8,28953.5,33815.8
setup_date,19910903,20021127,20071229,20030429,19990122
province,广东,上海,江苏,浙江,湖北
city,珠海市,上海市,苏州市,金华市,荆州市
website,www.yiduoli.com,www.csg.com.cn,www.chinasti.com,www.zjke.com/www.zjke.com.cn,www.feilihua.com


In [29]:
share_list = set(df_company_detail.ts_code.tolist())

### 上市公司管理层

|名称|类型|默认显示|描述|
|----|----|---|--|
|ts_code| str|Y| <font  color=#009688  >**TS股票代码**</font>|
|ann_date|str|Y| 公告日期|
|name|str|Y| <font  color=#009688  >**姓名**</font>|
|gender|str|Y| 性别|
|lev|str|Y| 岗位类别|
|title| str|Y| 岗位|
|edu|str|Y| 学历|
|national|str|Y| 国籍|
|birthday|str|Y| 出生年月|
|begin_date|str|Y| 上任日期|
|end_date|str|Y| 离任日期|
|resume|str|N| 个人简历|

用户需要2000积分才可以调取

In [33]:
def get_data(share_list,func,data_path='../data/',filename='temp.csv'):
    for i,comp in enumerate(share_list):
        df = func(ts_code=comp)
        if i == 0:
            df.to_csv(data_path + filename,index=False, mode='a')
        else:    
            df.to_csv(data_path + filename,index=False, mode='a', header=None)

        if i % 50 ==0: 
            print(comp,'_num_:',i)

        # 抱歉，您每分钟最多访问该接口80次
        time.sleep(1)

In [34]:
share_list = list(set(df_company_detail['ts_code']))

In [35]:
get_data(share_list,func = pro.stk_managers ,data_path = data_path ,filename=f'stk_managers.csv')

300797.SZ _num_: 0
002940.SZ _num_: 50
603019.SH _num_: 100
300454.SZ _num_: 150
600177.SH _num_: 200
002151.SZ _num_: 250
600489.SH _num_: 300
002842.SZ _num_: 350
300324.SZ _num_: 400
300711.SZ _num_: 450
300004.SZ _num_: 500
600009.SH _num_: 550
002923.SZ _num_: 600
600604.SH _num_: 650
300298.SZ _num_: 700
300448.SZ _num_: 750
300449.SZ _num_: 800
002688.SZ _num_: 850
300801.SZ _num_: 900
601901.SH _num_: 950
002631.SZ _num_: 1000
600485.SH _num_: 1050
600162.SH _num_: 1100
002487.SZ _num_: 1150
600804.SH _num_: 1200
600353.SH _num_: 1250
300812.SZ _num_: 1300
002750.SZ _num_: 1350
600999.SH _num_: 1400
002640.SZ _num_: 1450
000584.SZ _num_: 1500
300665.SZ _num_: 1550
300372.SZ _num_: 1600
600795.SH _num_: 1650
300475.SZ _num_: 1700
002238.SZ _num_: 1750
603869.SH _num_: 1800
603680.SH _num_: 1850
002439.SZ _num_: 1900
603980.SH _num_: 1950
300746.SZ _num_: 2000
600522.SH _num_: 2050
603706.SH _num_: 2100
002646.SZ _num_: 2150
002369.SZ _num_: 2200
300585.SZ _num_: 2250
603569.SH _

### 公募基金列表

https://tushare.pro/document/2?doc_id=19

|名称|类型|默认显示|描述|
|--|--|--|--|
|ts_code|str|Y|<font  color=#009688  >**基金代码**</font>|
|name|str|Y|简称|
|management|str|Y|<font  color=#009688  >**管理人**</font>|
|custodian|str|Y|<font  color=#009688  >**托管人**</font>|
|fund_type|str|Y|投资类型|
|found_date|str|Y|成立日期|
|due_date|str|Y|到期日期|
|list_date|str|Y|上市时间|
|issue_date|str|Y|发行日期|
|delist_date|str|Y|退市日期|
|issue_amount|float|Y|发行份额(亿)|
|m_fee|float|Y|管理费|
|c_fee|float|Y|托管费|
|duration_year|float|Y|存续期|
|p_value|float|Y|面值|
|min_amount|float|Y|起点金额(万元)|
|exp_return|float|Y|预期收益率|
|benchmark|str|Y|业绩比较基准|
|status|str|Y|存续状态D摘牌 I发行 L已上市|
|invest_type|str|Y|投资风格|
|type|str|Y|基金类型|
|trustee|str|Y|受托人|
|purc_startdate|str|Y|日常申购起始日|
|redm_startdate|str|Y|日常赎回起始日|
|market|str|Y|E场内O场外|

描述：获取公募基金数据列表，包括场内和场外基金
积分：用户需要1500积分才可以调取

In [36]:
df_e = pro.fund_basic(market='E')   # 交易市场: E场内 O场外（默认E）
df_o = pro.fund_basic(market='O')   # 交易市场: E场内 O场外（默认E）

df_all = pd.concat([df_e,df_o])

df_all.to_csv('../data/fund_basic.csv',index=False)

In [37]:
df_all.shape

(11087, 25)

In [38]:
df_all.head()

,ts_code,name,management,custodian,fund_type,found_date,due_date,list_date,issue_date,delist_date,...,min_amount,exp_return,benchmark,status,invest_type,type,trustee,purc_startdate,redm_startdate,market
0,159994.SZ,5GETF,银华基金,中国农业银行,股票型,20200122,None,20200228,20191227,None,...,0.1,None,中证5G通信主题指数,I,被动指数型,契约型开放式,None,20200228,20200228,E
1,515850.SH,证券FG,富国基金,中国建设银行,股票型,20200121,None,20200221,20191016,None,...,0.1,None,中证全指证券公司指数收益率,L,被动指数型,契约型开放式,None,20200221,20200221,E
2,159801.SZ,芯片基金,广发基金,中国工商银行,股票型,20200120,None,20200218,20200106,None,...,0.1,None,国证半导体芯片指数收益率,L,被动指数型,契约型开放式,None,20200218,20200218,E
3,159995.SZ,芯片ETF,华夏基金,中国建设银行,股票型,20200120,None,20200210,20191224,None,...,0.1,None,国证半导体芯片指数收益率,L,被动指数型,契约型开放式,None,20200210,20200210,E
4,515090.SH,可持续,博时基金,招商银行,股票型,20200119,None,20200220,20191216,None,...,0.1,None,中证可持续发展100指数收益率*95%+银行活期存款利率(税后)*5%,L,被动指数型,契约型开放式,None,20200220,20200220,E


### 公募基金公司


描述：获取公募基金管理人列表

积分：用户需要1500积分才可以调取，一次可以提取全部数据。

|名称|类型|默认显示|描述|
|--|--|--|--|
|name|str|Y|<font  color=#009688  >**基金公司名称**</font>|
|shortname|str|Y|简称|
|short_enname|str|N|英文缩写|
|province|str|Y|省份|
|city|str|Y|城市|
|address|str|Y|注册地址|
|phone|str|Y|电话|
|office|str|Y|办公地址|
|website|str|Y|公司网址|
|chairman|str|Y|法人代表|
|manager|str|Y|总经理|
|reg_capital|float|Y|注册资本|
|setup_date|str|Y|成立日期|
|end_date|str|Y|公司终止日期|
|employees|float|Y|员工总数|
|main_business|str|Y|主要产品及业务|
|org_code|str|Y|组织机构代码|
|credit_code|str|Y|统一社会信用代码|

In [39]:
df = pro.fund_company()

df.to_csv('../data/fund_company.csv', index=False)

In [40]:
df.head().T

,0,1,2,3,4
name,北京广能投资基金管理有限公司,宏源证券股份有限公司,国元证券股份有限公司,广发证券股份有限公司,长江证券股份有限公司
shortname,广能基金,宏源证券,国元证券,广发证券,长江证券
province,北京,新疆,安徽,广东,湖北
city,北京市,乌鲁木齐市,合肥市,广州市,武汉市
address,北京市朝阳区北四环中路27号院5号楼2712-2715A,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,安徽省合肥市梅山路18号,广东省广州市黄埔区中新广州知识城腾飞一街2号618室,湖北省武汉市江汉区新华路特8号
phone,None,86-991-2301870,"86-551-62207323,86-551-62207968","86-20-87555888,86-20-87550565,86-20-87550265","86-27-65799866,86-27-65799856"
office,北京市朝阳区北四环中路27号院5号楼2712-2715A,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,安徽省合肥市梅山路18号,"广东省广州市天河区天河北路183-187号大都会广场40楼5楼,7楼,8楼,18楼,19楼,...",湖北省武汉市江汉区新华路特8号
website,www.gnfund.cn,www.hysec.com,www.gyzq.com.cn,www.gf.com.cn,www.cjsc.com
chairman,刘锡潜,冯戎,蔡咏,孙树明,尤习贵
manager,杨运成,冯戎,俞仕新,林治海,刘元瑞


### 关系——公募基金持仓数据

获取公募基金持仓数据，**季度**更新
积分：用户需要至少1000积分才可以调取，

https://tushare.pro/document/2?doc_id=121

|名称|类型|默认显示|描述|
|--|--|--|--|
|ts_code| str| Y| <font  color=#009688  >**TS基金代码**</font>|
|ann_date|str |Y| 公告日期|
|end_date|str |Y| 截止日期|
|symbol|str| Y|  <font  color=#009688  >**股票代码**</font>|
|mkv |float| Y| 持有股票市值(元)|
|amount|float| Y| 持有股票数量（股）|
|stk_mkv_ratio| float| Y| 占股票市值比|
|stk_float_ratio| float| Y| 占流通股本比例|

In [41]:
fund_list = list(set(df_all.ts_code))

In [42]:
len(fund_list)

11087

In [43]:
fund_list[:5]

['159805.OF', '161818.OF', '007368.OF', '005866.OF', '150029.SZ']

In [44]:
get_data(fund_list,func = pro.fund_portfolio ,data_path = data_path ,filename=f'fund_portfolio.csv')

159805.OF _num_: 0
004532.OF _num_: 50
001502.OF _num_: 100
501087.OF _num_: 150
310388.OF _num_: 200
002812.OF _num_: 250
007387.OF _num_: 300
004841.OF _num_: 350
006295.OF _num_: 400
007127.OF _num_: 450
006050.OF _num_: 500
005549.OF _num_: 550
005229.OF _num_: 600
160311.SZ _num_: 650
003188.OF _num_: 700
167704.OF _num_: 750
500015.SH _num_: 800
005301.OF _num_: 850
000575.OF _num_: 900
006391.OF _num_: 950
001840.OF _num_: 1000
002645.OF _num_: 1050
002104.OF _num_: 1100
006202.OF _num_: 1150
005417.OF _num_: 1200
159952.SZ _num_: 1250
002739.OF _num_: 1300
002376.OF _num_: 1350
005576.OF _num_: 1400
006195.OF _num_: 1450
005490.OF _num_: 1500
005800.OF _num_: 1550
008512.OF _num_: 1600
007946.OF _num_: 1650
008192.OF _num_: 1700
008018.OF _num_: 1750
005019.OF _num_: 1800
002057.OF _num_: 1850
001105.OF _num_: 1900
002986.OF _num_: 1950
000298.OF _num_: 2000
007120.OF _num_: 2050
004648.OF _num_: 2100
519779.OF _num_: 2150
070015.OF _num_: 2200
020003.OF _num_: 2250
007920.OF _

## 构建数据模型 


通过对获取的数据进行梳理，明确内部之间的关联关系，确定节点和关系和节点

实体(节点)方面
1. 省份
2. 城市
3. 公司(上市公司 \ 基金管理公司 \ 基金托管公司)
4. 人（上市公司管理层）
5. 基金
6. 行业

关系方面
1. 城市 - [IN_PROVINCE] - > 省份 
2. 公司(上市公司\基金管理公司\ 基金托管公司) - [IN_CITY] - > 城市
4. 公司(上市公司) - [HAS_MANGER] - > 人（上市公司管理层）  [6类]
5. 公司(上市公司) - [IN_INDUSTRY] - > 行业
6. 基金  - [HAS_MANAGEMNET] - > 公司(基金管理公司)
6. 基金  - [HAS_CUSTODIAN] - > 公司(基金托管公司)
7. 基金  - [IN_PORTFOLIO] ->  公司(上市公司)

初步确定数据模型如下。

![shuju](../pictures/screenshot_2.png)